In [ ]:
import pandas as pd

df = pd.read_csv('../data/bruto/crimes_violentos_2023.csv', sep= ';')
df[df["NO_UF"] == "Minas Gerais"]

print(df)


C:\Users\Meu computador\AppData\Local\Temp\ipykernel_11328\1319323128.py:3: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/bruto/microdados_ed_basica_2024.csv', sep= ';', encoding='latin1')


        NU_ANO_CENSO     NO_REGIAO  CO_REGIAO             NO_UF SG_UF  CO_UF  \
0               2024         Norte          1          Rondônia    RO     11   
1               2024         Norte          1          Rondônia    RO     11   
2               2024         Norte          1          Rondônia    RO     11   
3               2024         Norte          1          Rondônia    RO     11   
4               2024         Norte          1          Rondônia    RO     11   
...              ...           ...        ...               ...   ...    ...   
215540          2024  Centro-Oeste          5  Distrito Federal    DF     53   
215541          2024  Centro-Oeste          5  Distrito Federal    DF     53   
215542          2024  Centro-Oeste          5  Distrito Federal    DF     53   
215543          2024  Centro-Oeste          5  Distrito Federal    DF     53   
215544          2024  Centro-Oeste          5  Distrito Federal    DF     53   

                 NO_MUNICIPIO  CO_MUNIC